In [159]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [160]:
# Load dataset
df = pd.read_csv('dataset/Housing.csv')

# Replace 'yes'/'no' with 1/0
df[['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']] = df[['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']].replace({'yes': 1, 'no': 0}).astype(int)

# Encoding 'furnishingstatus'
from sklearn.preprocessing import LabelEncoder
df['furnishingstatus'] = LabelEncoder().fit_transform(df['furnishingstatus'])

# Split features (X) and target (y)
X = df.drop("price", axis=1)
y = df["price"]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print(f'Data training size: {len(X_train)}')
print(f'Data testing size: {len(X_test)}')

Data training size: 436
Data testing size: 109


C:\Users\bramm\AppData\Local\Temp\ipykernel_4480\967502231.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']] = df[['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']].replace({'yes': 1, 'no': 0}).astype(int)


In [161]:
# Scaling the input features using MinMaxScaler
scaler_X = MinMaxScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Scale the target (price) using MinMaxScaler
scaler_y = MinMaxScaler()
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

# Check the scaled data
print(X_train_scaled[:5])
print(y_train_scaled[:5])

[[0.29896907 0.4        0.33333333 1.         1.         0.
  0.         0.         1.         0.33333333 0.         0.        ]
 [0.3814433  0.4        0.33333333 0.         1.         0.
  1.         0.         1.         1.         0.         0.5       ]
 [0.14886598 0.2        0.         0.         1.         0.
  1.         0.         1.         0.66666667 0.         0.        ]
 [0.06597938 0.4        0.         0.33333333 1.         0.
  1.         0.         0.         0.         1.         1.        ]
 [0.1443299  0.4        0.         0.33333333 1.         0.
  0.         0.         0.         0.         0.         1.        ]]
[0.55       0.43333333 0.20666667 0.16       0.12      ]


In [162]:
# Activation functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return np.where(x > 0, 1, 0)

In [163]:
# Loss functions
def mse_loss(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def mse_loss_derivative(y_true, y_pred):
    return 2 * (y_pred - y_true) / y_true.size

def mae_loss(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def mae_loss_derivative(y_true, y_pred):
    return np.where(y_pred > y_true, 1, -1) / y_true.size

def initialize_weights(input_size, hidden_size, output_size):
    np.random.seed(42)
    W1 = np.random.randn(input_size, hidden_size) * 0.001
    b1 = np.zeros((1, hidden_size))
    W2 = np.random.randn(hidden_size, output_size) * 0.001
    b2 = np.zeros((1, output_size))
    return W1, b1, W2, b2


In [164]:
def forward_pass(X, W1, b1, W2, b2):
    Z1 = np.dot(X, W1) + b1 
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2 
    A2 = Z2  
    return Z1, A1, Z2, A2

def backward_pass(X, y, Z1, A1, Z2, A2, W1, W2, b1, b2, loss_derivative, lr):
    m = y.shape[0]

    y = y.reshape(-1, 1)
    
    dZ2 = loss_derivative(y, A2)

    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)

    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    
    return W1, b1, W2, b2


In [165]:
def train(X_train, y_train, X_test, y_test, hidden_size, epochs, lr, batch_size=None, loss_func=mse_loss, loss_derivative_func=mse_loss_derivative, method='batch'):
    input_size = X_train.shape[1]
    output_size = 1
    W1, b1, W2, b2 = initialize_weights(input_size, hidden_size, output_size)

    train_losses = []
    test_losses = []

    for epoch in range(epochs + 1):
        if method == 'stochastic':
            indices = np.random.permutation(X_train.shape[0])
            for i in indices:
                X_batch = X_train[i:i+1]
                y_batch = y_train[i:i+1]
                Z1, A1, Z2, A2 = forward_pass(X_batch, W1, b1, W2, b2)
                W1, b1, W2, b2 = backward_pass(X_batch, y_batch, Z1, A1, Z2, A2, W1, W2, b1, b2, loss_derivative_func, lr)

        elif method == 'minibatch' and batch_size is not None:
            indices = np.random.permutation(X_train.shape[0])
            for i in range(0, X_train.shape[0], batch_size):
                batch_indices = indices[i:i+batch_size]
                X_batch = X_train[batch_indices]
                y_batch = y_train[batch_indices]
                Z1, A1, Z2, A2 = forward_pass(X_batch, W1, b1, W2, b2)
                W1, b1, W2, b2 = backward_pass(X_batch, y_batch, Z1, A1, Z2, A2, W1, W2, b1, b2, loss_derivative_func, lr)

        else: 
            Z1, A1, Z2, A2 = forward_pass(X_train, W1, b1, W2, b2)
            W1, b1, W2, b2 = backward_pass(X_train, y_train, Z1, A1, Z2, A2, W1, W2, b1, b2, loss_derivative_func, lr)

        train_predictions = forward_pass(X_train, W1, b1, W2, b2)[-1]
        train_loss = loss_func(y_train, train_predictions)
        train_losses.append(train_loss)

        test_predictions = forward_pass(X_test, W1, b1, W2, b2)[-1]
        test_loss = loss_func(y_test, test_predictions)
        test_losses.append(test_loss)

        if epoch % 10 == 0 or epoch == epochs:
            print(f'Epoch {epoch}, Training Loss: {train_loss}, Test Loss: {test_loss}')

    return W1, b1, W2, b2, train_losses, test_losses

In [166]:
def test(X, y, W1, b1, W2, b2, loss_func):
    _, _, _, predictions = forward_pass(X, W1, b1, W2, b2)
    loss = loss_func(y, predictions)
    return loss, predictions

In [167]:
def run_experiments(X_train, y_train, X_test, y_test, hidden_size=5, epochs=500, lr=0.001, batch_size=64):
    results = {}

    loss_functions = {'MSE': (mse_loss, mse_loss_derivative), 'MAE': (mae_loss, mae_loss_derivative)}
    methods = ['batch', 'stochastic', 'minibatch']

    for loss_name, (loss_func, loss_derivative_func) in loss_functions.items():
        for method in methods:
            print(f"\nRunning experiment with {loss_name} and {method} gradient descent")
            
            W1, b1, W2, b2, train_losses, test_losses = train(X_train, y_train, X_test, y_test, hidden_size, epochs, lr, batch_size, loss_func, loss_derivative_func, method)
            
            final_train_loss = train_losses[-1]
            final_test_loss, _ = test(X_test, y_test, W1, b1, W2, b2, loss_func)
            results[f'{loss_name}_{method}'] = (final_train_loss, final_test_loss)
            
            print(f"Final Training Loss: {final_train_loss:.3f}")
            print(f"Final Test Loss: {final_test_loss:.3f}")

    print("\nSummary Results:")
    for key, (train_loss, test_loss) in results.items():
        print(f"{key.upper()} - Training Loss: {train_loss:.3f}, Test Loss: {test_loss:.3f}")

    return results

In [168]:
experiment_results = run_experiments(X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, hidden_size=50, epochs=50, lr=0.001, batch_size=8)


Running experiment with MSE and batch gradient descent
Epoch 0, Training Loss: 0.1072471519423707, Test Loss: 0.14209167714597784
Epoch 10, Training Loss: 0.10723987803778846, Test Loss: 0.14208366263771427
Epoch 20, Training Loss: 0.10723260480068106, Test Loss: 0.14207564883085713
Epoch 30, Training Loss: 0.10722533223090844, Test Loss: 0.14206763572529868
Epoch 40, Training Loss: 0.10721806032838425, Test Loss: 0.14205962332121946
Epoch 50, Training Loss: 0.10721078909290664, Test Loss: 0.14205161161847918
Final Training Loss: 0.107
Final Test Loss: 0.142

Running experiment with MSE and stochastic gradient descent
Epoch 0, Training Loss: 0.04186460782624392, Test Loss: 0.06732452735090676
Epoch 10, Training Loss: 0.02796945357184964, Test Loss: 0.04658579592932605
Epoch 20, Training Loss: 0.027968390380342798, Test Loss: 0.046729074637523495
Epoch 30, Training Loss: 0.02796750775782698, Test Loss: 0.04665006875835159
Epoch 40, Training Loss: 0.027972300469214527, Test Loss: 0.0467